# Demo Notebook for Creating a Reboot Motion Custom Report from a File

__[CoLab Notebook Link](https://githubtocolab.com/RebootMotion/reboot-toolkit/blob/main/examples_api/RebootMotionCustomReport.ipynb)__

Run the cells in order, making sure to enter your API KEY in the cell when prompted

In [ ]:
#@title Install Python Libraries

!pip install pandas
!pip install requests

In [ ]:
#@title Import Python Libraries

import pandas as pd
import requests

from getpass import getpass

In [ ]:
#@title Set your Reboot Motion API Key

# If you don't have an API KEY already, log in to the dashboard and create one, 
# then paste your API KEY below when prompted
API_KEY = getpass('Input Reboot Motion API Key:')

API_HEADER = {'x-api-key': API_KEY}
API_URL_SEGMENTS = "https://api.rebootmotion.com/player_group_segments"
API_URL_REQUESTED_ANALYSES = "https://api.rebootmotion.com/requested_analyses"

In [ ]:
#@title Set Player and Play Metadata for the Custom Report

# Set the name for the report
# We recommend using the format "primary segment label vs comparison segment label"
REPORT_TITLE = 'Player Pitch Type 1 vs Pitch Type 2'

# Set the dominant hand for the analysis
DOM_HAND = 'RHA'  # either RHA or LHA

# Reboot Motion's movement type ID
# 1 = baseball-hitting
# 2 = baseball-pitching
MOVEMENT_TYPE_ID = 2  

In [ ]:
#@title Upload File with Play IDs and a Column for Filtering

# 1) Upload a CSV or Excel file with a column of mlb play GUIDs

# 2) Type the filename for the uploaded 
custom_metadata_filename = ''

# 3) Set the name of the column with the mlb play GUID
metadata_play_id_name = 'guid'

# 4) Set the name of the column with categories for filtering the primary and comparison segments
col_for_filtering = 'pitch_type'

if len(custom_metadata_filename.strip()) > 0:
    if custom_metadata_filename.endswith('.xlsx'):
        df = pd.read_excel(custom_metadata_filename)
    
    elif custom_metadata_filename.endswith('.csv'):
        df = pd.read_csv(custom_metadata_filename)
    
    else:
        raise ValueError('Unsupported file type - only CSVs or Excel files are accepted')

else:
    raise FileNotFoundError('Upload a CSV or Excel file and type the name above')

In [ ]:
#@title Create the Primary Analysis Segment

# list of column values to filter on from the uploaded file for the primary segment
primary_segment_vals_to_filter_on = {'pitch type 1'}

primary_segment_movements = list(
    df.loc[df[col_for_filtering].isin(primary_segment_vals_to_filter_on)][metadata_play_id_name].unique()
)

primary_segment_criteria = {
    "external_context_ids": primary_segment_movements,
    "movement_type_id": MOVEMENT_TYPE_ID,
    "dom_hand": DOM_HAND
}

create_primary_segment = requests.post(
    API_URL_SEGMENTS,
    headers=API_HEADER,
    json=primary_segment_criteria
)

primary_segment_id = create_primary_segment.json()["analysis_segment_id"]

print('Primary analysis segment created with ID', primary_segment_id)

In [ ]:
#@title Create the Comparison Analysis Segment

# list of column values to filter on from the uploaded file for the primary segment
comparison_segment_vals_to_filter_on = {'pitch type 2'}

comparison_segment_movements = list(
    df.loc[df[col_for_filtering].isin(comparison_segment_vals_to_filter_on)][metadata_play_id_name].unique()
)
        
comparison_segment_criteria = {
    "external_context_ids": comparison_segment_movements,
    "movement_type_id": MOVEMENT_TYPE_ID,
    "dom_hand": DOM_HAND
}

create_comparison_segment = requests.post(
    API_URL_SEGMENTS,
    headers=API_HEADER,
    json=comparison_segment_criteria
)

comparison_segment_id = create_comparison_segment.json()["analysis_segment_id"]

print('Comparison analysis segment created with ID', comparison_segment_id)

In [ ]:
#@title Submit the Custom Analysis Request

requested_analysis_criteria = {
    "name": REPORT_TITLE,
    "primary_analysis_segment_id": int(primary_segment_id),
    "comparison_analysis_segment_id": int(comparison_segment_id),
    "status": "requested"
}

create_requested_analysis = requests.post(
    API_URL_REQUESTED_ANALYSES,
    headers=API_HEADER,
    json=requested_analysis_criteria
)

print("Submitted Requested Analysis Request!")
print("Log in to your Dashboard and the Request will show up under the Analyze Tab")
print()
print('Below is the API Response (if there was an error, contact Reboot Motion support)')
print(create_requested_analysis.json())